# Using Djed in Marlowe Contracts

In this example, Christopher Marlowe sells the Swan Theatre to Francis Beaumont for 2250 `DjedUSD`. The Swan Theatre is represented as an NFT.

Make a note of the versions of the Marlowe repository and software.

In [ ]:
marlowe-cli --version

marlowe-cli 0.0.5.0


In [ ]:
cardano-cli --version

cardano-cli 1.34.1 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [ ]:
git rev-parse HEAD

de3ca53f994f989ba7f5693e5e03bdfbcd52beee


## Select the Cardano Network.

We use the public `testnet` for this example.

In [1]:
export CARDANO_TESTNET_MAGIC=1097911063
export CARDANO_NODE_SOCKET_PATH=node.socket

## Parties to the Contract

### Christopher Marlowe

Use the keys and address for Christopher Marlowe.

In [2]:
MARLOWE_SKEY=christopher-marlowe.skey
MARLOWE_VKEY=christopher-marlowe.vkey
MARLOWE_PKH=$(cardano-cli address key-hash --payment-verification-key-file $MARLOWE_VKEY)
MARLOWE_ADDR=$(cat christopher-marlowe.testnet.address)
echo $MARLOWE_ADDR

addr_test1vrssw4edcts00kk6lp7p5n64666m23tpprqaarmdwkaq69gfvqnpz


Christopher has the requisite UTxOs at his address.

In [3]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $MARLOWE_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
a2cb791acb841e28460f1b3b0c69b9ebdf853cfaa08eb05bb3f324fca1bab46d     0        490361492 lovelace + TxOutDatumNone
a2cb791acb841e28460f1b3b0c69b9ebdf853cfaa08eb05bb3f324fca1bab46d     1        2000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.5377616e + TxOutDatumNone


We decode the hexadecimal value in the second UTxO to see that it is the Swan Theatre token.

In [4]:
echo -n 5377616e | tr '[:lower:]' '[:upper:]' | basenc --decode --base16

Swan

Keep a record of the UTxOs, so that we can spend them later.

In [5]:
TX_0_MARLOWE_ADA="a2cb791acb841e28460f1b3b0c69b9ebdf853cfaa08eb05bb3f324fca1bab46d#0"
TX_0_MARLOWE_SWAN="a2cb791acb841e28460f1b3b0c69b9ebdf853cfaa08eb05bb3f324fca1bab46d#1"

Also record the full identifier for the Swan Theatre token.

In [6]:
SWAN=8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.Swan

### Francis Beaumont

Use the keys and address for Francis Beaumont.

In [7]:
BEAUMONT_SKEY=francis-beaumont.skey
BEAUMONT_VKEY=francis-beaumont.vkey
BEAUMONT_PKH=$(cardano-cli address key-hash --payment-verification-key-file $BEAUMONT_VKEY)
BEAUMONT_ADDR=$(cat francis-beaumont.testnet.address)
echo $BEAUMONT_ADDR

addr_test1vzzpzll6gsl9npf8wfhk2zg8sy2we50jcqc7w8w46gua2pqq7cw2q


Francis also has the requisite UTxOs at his address.

In [8]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $BEAUMONT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
a2cb791acb841e28460f1b3b0c69b9ebdf853cfaa08eb05bb3f324fca1bab46d     2        2000000 lovelace + 2250 f4cf384ddd1b1377b08302b17990e9618b62924f5705458c17ee4f7d.446a6564555344 + TxOutDatumNone
c86ef2ac159794ce00c0c1dce72149a065a3b559da7f0f5398abd629795fef60     0        500224541 lovelace + TxOutDatumNone


We decode the hexadecimal value in the first UTxO to see that they are `DjedUSD` tokens.

In [9]:
echo -n 446a6564555344 | tr '[:lower:]' '[:upper:]' | basenc --decode --base16

DjedUSD

In [10]:
TX_0_BEAUMONT_ADA="c86ef2ac159794ce00c0c1dce72149a065a3b559da7f0f5398abd629795fef60#0"
TX_0_BEAUMONT_DJED="a2cb791acb841e28460f1b3b0c69b9ebdf853cfaa08eb05bb3f324fca1bab46d#2"

Also record the full identifier for the `DjedUSD` tokens.

In [11]:
DJED=f4cf384ddd1b1377b08302b17990e9618b62924f5705458c17ee4f7d.DjedUSD

## The Marlowe Contract

The price of the Swan Theatre is 2250 `DjedUSD`.

In [12]:
PRICE=2250

Compute deadlines relative to now.

In [13]:
MILLISECOND=1000
HOUR=$((60*60*MILLISECOND))
NOW=$(($(date -u +%s)*MILLISECOND))
echo $NOW

1652847057000


In [14]:
DEADLINE_A=$((NOW+12*HOUR))
echo $DEADLINE_A

1652890257000


In [15]:
DEADLINE_B=$((NOW+24*HOUR))
echo $DEADLINE_B

1652933457000


Create the contract from a template for swaps.

In [16]:
marlowe-cli template swap --minimum-ada 2000000             \
                          --a-party PK=$MARLOWE_PKH         \
                          --a-token $SWAN                   \
                          --a-amount 1                      \
                          --a-timeout $DEADLINE_A           \
                          --b-party PK=$BEAUMONT_PKH        \
                          --b-token $DJED                   \
                          --b-amount $PRICE                 \
                          --b-timeout $DEADLINE_B           \
                          --out-contract-file tx-1.contract \
                          --out-state-file    tx-1.state

Inspect the contract.

In [17]:
json2yaml tx-1.contract

timeout: 1652890257000
timeout_continuation: close
when:
- case:
    deposits: 1
    into_account:
      pk_hash: e107572dc2e0f7dadaf87c1a4f55d6b5b5456108c1de8f6d75ba0d15
    of_token:
      currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
      token_name: Swan
    party:
      pk_hash: e107572dc2e0f7dadaf87c1a4f55d6b5b5456108c1de8f6d75ba0d15
  then:
    timeout: 1652933457000
    timeout_continuation: close
    when:
    - case:
        deposits: 2250
        into_account:
          pk_hash: 84117ffa443e598527726f6509078114ecd1f2c031e71dd5d239d504
        of_token:
          currency_symbol: f4cf384ddd1b1377b08302b17990e9618b62924f5705458c17ee4f7d
          token_name: DjedUSD
        party:
          pk_hash: 84117ffa443e598527726f6509078114ecd1f2c031e71dd5d239d504
      then:
        from_account:
          pk_hash: e107572dc2e0f7dadaf87c1a4f55d6b5b5456108c1de8f6d75ba0d15
        pay: 1
        then:
          from_account:
            pk_hash: 84117ffa443e

Inspect the initial state.

In [18]:
json2yaml tx-1.state

accounts:
- - - pk_hash: e107572dc2e0f7dadaf87c1a4f55d6b5b5456108c1de8f6d75ba0d15
    - currency_symbol: ''
      token_name: ''
  - 2000000
boundValues: []
choices: []
minTime: 1


## Running the Contract on the Public Testnet

### Transaction 1. Christopher Creates the Contract

First we initialize the `.marlowe` file that bundles the information needed to transition a contract forward.

In [19]:
marlowe-cli run initialize --contract-file  tx-1.contract \
                           --state-file     tx-1.state    \
                           --out-file       tx-1.marlowe

Submit the transaction to the blockchain.

In [20]:
marlowe-cli run execute --tx-in $TX_0_MARLOWE_ADA       \
                        --required-signer $MARLOWE_SKEY \
                        --marlowe-out-file tx-1.marlowe \
                        --change-address $MARLOWE_ADDR  \
                        --out-file /dev/null            \
                        --submit 600                    \
                        --print-stats


Fee: Lovelace 202197
Size: 821 / 16384 = 5%
Execution units:
  Memory: 0 / 16000000 = 0%
  Steps: 0 / 10000000000 = 0%
TxId "67689604bc6232c04f7f0a544e683bd2889fb2053a91622c29364948794c12e9"


Record the transaction ID, which we need for spending UTxOs from the transaction.

In [21]:
TX_1=67689604bc6232c04f7f0a544e683bd2889fb2053a91622c29364948794c12e9

Compute the Marlowe contract's validator address.

In [22]:
VALIDATOR_ADDR=$(jq -r '.marloweValidator.address' tx-1.marlowe)
echo $VALIDATOR_ADDR

addr_test1wquea223tl4cdz6n6w000g84hznt2tedafkdvwr6njhex9q39w5zm


Query the validator's address to see that the contract was created.

In [23]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $VALIDATOR_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
67689604bc6232c04f7f0a544e683bd2889fb2053a91622c29364948794c12e9     1        2000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "194a45b9721b605749ec09f3bd370a3cc3713193375688e05b3c28ebad7a0e06"


See that the balance of funds at Christopher's address.

In [24]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $MARLOWE_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
67689604bc6232c04f7f0a544e683bd2889fb2053a91622c29364948794c12e9     0        488159295 lovelace + TxOutDatumNone
a2cb791acb841e28460f1b3b0c69b9ebdf853cfaa08eb05bb3f324fca1bab46d     1        2000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.5377616e + TxOutDatumNone


### Transaction 2. Christopher Deposits the NFT Representing Ownership of the Swan Theatre

Now we transition from the initial `.marlowe` file to the next `.marlowe` file by depositing 1 `Swan` token.

In [25]:
marlowe-cli run prepare --marlowe-file tx-1.marlowe         \
                        --deposit-account PK=$MARLOWE_PKH   \
                        --deposit-party PK=$MARLOWE_PKH     \
                        --deposit-token $SWAN               \
                        --deposit-amount 1                  \
                        --invalid-before $NOW               \
                        --invalid-hereafter $((NOW+6*HOUR)) \
                        --out-file tx-2.marlowe

Submit the transaction to the blockchain.

In [26]:
marlowe-cli run execute --tx-in $TX_0_MARLOWE_SWAN      \
                        --tx-in "$TX_1#0"               \
                        --required-signer $MARLOWE_SKEY \
                        --marlowe-in-file tx-1.marlowe  \
                        --tx-in-marlowe "$TX_1#1"       \
                        --tx-in-collateral "$TX_1#0"    \
                        --marlowe-out-file tx-2.marlowe \
                        --change-address $MARLOWE_ADDR  \
                        --out-file /dev/null            \
                        --submit 600                    \
                        --print-stats


Fee: Lovelace 1170290
Size: 14041 / 16384 = 85%
Execution units:
  Memory: 4538228 / 16000000 = 28%
  Steps: 1603062306 / 10000000000 = 16%
TxId "5213dbf8ad4bd9b0bd36883e332531cd59dc8cf7ecd03910bdd8ed96860c4e9e"


Record the transaction ID, which we need for spending UTxOs from the transaction.

In [27]:
TX_2=5213dbf8ad4bd9b0bd36883e332531cd59dc8cf7ecd03910bdd8ed96860c4e9e

Query the validator's address to see that the contract received the deposit.

In [28]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $VALIDATOR_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5213dbf8ad4bd9b0bd36883e332531cd59dc8cf7ecd03910bdd8ed96860c4e9e     1        2000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.5377616e + TxOutDatumHash ScriptDataInAlonzoEra "6556b935e0a345df25aa47470aee5c7912a200aae7d21324924d4b9a93d477d9"


See that the balance of funds at Christopher's address.

In [29]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $MARLOWE_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5213dbf8ad4bd9b0bd36883e332531cd59dc8cf7ecd03910bdd8ed96860c4e9e     0        488989005 lovelace + TxOutDatumNone


### Transaction 3. Francis Deposits the Djed to Pay for the Swan Theatre, Closing the Contract

We again transition from one `.marlowe` file to the next `.marlowe` file by depositing the `DjedUSD`. This causes the contract to makes its payments to all parties and then close.

In [30]:
marlowe-cli run prepare --marlowe-file tx-2.marlowe         \
                        --deposit-account PK=$BEAUMONT_PKH  \
                        --deposit-party PK=$BEAUMONT_PKH    \
                        --deposit-token $DJED               \
                        --deposit-amount $PRICE             \
                        --invalid-before $NOW               \
                        --invalid-hereafter $((NOW+6+HOUR)) \
                        --out-file tx-3.marlowe

Payment 1
  Acccount: PK "e107572dc2e0f7dadaf87c1a4f55d6b5b5456108c1de8f6d75ba0d15"
  Payee: Party (PK "84117ffa443e598527726f6509078114ecd1f2c031e71dd5d239d504")
  Ada: 0.000000
  8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d."Swan": 1
Payment 2
  Acccount: PK "84117ffa443e598527726f6509078114ecd1f2c031e71dd5d239d504"
  Payee: Party (PK "e107572dc2e0f7dadaf87c1a4f55d6b5b5456108c1de8f6d75ba0d15")
  Ada: 0.000000
  f4cf384ddd1b1377b08302b17990e9618b62924f5705458c17ee4f7d."DjedUSD": 2250
Payment 3
  Acccount: PK "e107572dc2e0f7dadaf87c1a4f55d6b5b5456108c1de8f6d75ba0d15"
  Payee: Party (PK "e107572dc2e0f7dadaf87c1a4f55d6b5b5456108c1de8f6d75ba0d15")
  Ada: 2.000000


Submit the transaction to the blockchain.

In [31]:
marlowe-cli run execute --tx-in $TX_0_BEAUMONT_DJED           \
                        --tx-in $TX_0_BEAUMONT_ADA            \
                        --required-signer $BEAUMONT_SKEY      \
                        --marlowe-in-file tx-2.marlowe        \
                        --tx-in-marlowe "$TX_2#1"             \
                        --tx-in-collateral $TX_0_BEAUMONT_ADA \
                        --marlowe-out-file tx-3.marlowe       \
                        --change-address $BEAUMONT_ADDR       \
                        --out-file /dev/null                  \
                        --submit 600                          \
                        --print-stats


Fee: Lovelace 1258245
Size: 13503 / 16384 = 82%
Execution units:
  Memory: 5959514 / 16000000 = 37%
  Steps: 2013861504 / 10000000000 = 20%
TxId "673f2019dcba6950955f35f2c2debdc9e6154838b52e89663dfce5f2d420ff50"


Record the transaction ID, which we need for spending UTxOs from the transaction.

In [32]:
TX_3=673f2019dcba6950955f35f2c2debdc9e6154838b52e89663dfce5f2d420ff50

There are not funds in the contract any longer.

In [33]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $VALIDATOR_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


Christopher has received the 2250 `DjedUSD` in return for the Swan Theatre.

In [34]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $MARLOWE_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5213dbf8ad4bd9b0bd36883e332531cd59dc8cf7ecd03910bdd8ed96860c4e9e     0        488989005 lovelace + TxOutDatumNone
673f2019dcba6950955f35f2c2debdc9e6154838b52e89663dfce5f2d420ff50     2        2000000 lovelace + 2250 f4cf384ddd1b1377b08302b17990e9618b62924f5705458c17ee4f7d.446a6564555344 + TxOutDatumNone


Francis Beaumont has received the NFT representing ownership of the Swan Theatre.

In [35]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $BEAUMONT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
673f2019dcba6950955f35f2c2debdc9e6154838b52e89663dfce5f2d420ff50     0        499621498 lovelace + TxOutDatumNone
673f2019dcba6950955f35f2c2debdc9e6154838b52e89663dfce5f2d420ff50     1        1344798 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.5377616e + TxOutDatumNone


## Recap

The three transactions above are viewable on Cardanoscan:

1. Christopher creates the contract: [67689604bc6232c04f7f0a544e683bd2889fb2053a91622c29364948794c12e9](https://testnet.cardanoscan.io/transaction/67689604bc6232c04f7f0a544e683bd2889fb2053a91622c29364948794c12e9)
1. Christopher deposits the Swan token: [5213dbf8ad4bd9b0bd36883e332531cd59dc8cf7ecd03910bdd8ed96860c4e9e](https://testnet.cardanoscan.io/transaction/5213dbf8ad4bd9b0bd36883e332531cd59dc8cf7ecd03910bdd8ed96860c4e9e)
3. Christopher creates the contract: [673f2019dcba6950955f35f2c2debdc9e6154838b52e89663dfce5f2d420ff50](https://testnet.cardanoscan.io/transaction/673f2019dcba6950955f35f2c2debdc9e6154838b52e89663dfce5f2d420ff50)